In [6]:
import os 
import tqdm
import imageio
import numpy as np
import time
import random
import math
import scipy.io as sio
from PIL import Image#

In [7]:

fx = 365.481
fy = 365.481
cx = 257.346
cy = 210.347
 

SAMPLE_NUM = 2048
fps_sample_num=512
K = 20  # max frame limit for temporal rank

save_path = '../T'


In [8]:
try:
    os.makedirs(save_path)
except OSError:
    pass

In [9]:
def main():
    data_path = 'NTU_png/train'
    sub_Files = os.listdir(data_path)
    sub_Files.sort()
    #print(sub_Files[1:])
    index17=0

    for s_fileName in sub_Files[1:]:#lx#Traverse 17 settings folders
        index17=index17+1
        #print(s_fileName)
        if(index17>0):
            videoPath = os.path.join(data_path, s_fileName)
            #print(videoPath)
            #videoPath = os.path.join(data_path, s_fileName, 'nturgb+d_depth_masked')
            if os.path.isdir(videoPath):#lx#Determine whether it is a directory
                print(s_fileName)
                video_Files = os.listdir(videoPath)
                video_Files.sort()
                #print(video_Files[1:])
                video_index=0
                video_num=len(video_Files[1:])
                #print(time.time())
                for video_FileName in video_Files[1:]:#lx#Traverse the sample folder
                    video_index=video_index+1
                    #print(video_index)
                    
                    filename=video_FileName+'.npy'
                    file = os.path.join(save_path,filename)  #保存文件名
                    #print(file)
                    pngPath=os.path.join(videoPath,video_FileName)
                    #print(pngPath)
                    imgNames=os.listdir(pngPath)
                    imgNames.sort()
                    #print(imgNames)
                    
                    #select a fixed number K of images
                    n_frame=len(imgNames)
                    print(n_frame)
                    all_sam=np.arange(n_frame)
                    #print(all_sam)
                    
                    #Randomly sampled frames   随机采样
                    if(1==0):
                        if n_frame>K:
                            frame_index=random.sample(list(all_sam),K)
                            n_frame=K
                        else:
                            if n_frame<K/2:
                                frame_index=random.sample(list(all_sam),K-n_frame-n_frame)
                                frame_index.extend(list(all_sam))
                                frame_index.extend(list(all_sam))
                                n_frame=K
                            else:
                                frame_index=random.sample(list(all_sam),K-n_frame)
                                frame_index.extend(list(all_sam))
                                n_frame=K
                            frame_index=all_sam.tolist()
                    #Randomly sample frames at equal intervals   stratified sampling分层采样
                    if(1==0):
                        print('now')
                        if n_frame>K:
                            frame_index=[]
                            for jj in range(K):
                                iii=int(np.random.randint(int(n_frame*jj/K),int(n_frame*(jj+1)/K),size=1))
                                frame_index.append(iii)
                            n_frame=K
                        else:
                            frame_index=random.sample(list(all_sam),K-n_frame)
                            frame_index.extend(list(all_sam))
                            n_frame=K
                    #Sampling frames at equal intervals    等间隔采样
                    if(1==1):
                        if n_frame>K:
                            frame_index=[]
                            for jj in range(K):
                                iii= int((int(n_frame*jj/K)+int(n_frame*(jj+1)/K))/2)
                                frame_index.append(iii)
                            n_frame=K
                        else:
                            frame_index=random.sample(list(all_sam),K-n_frame)
                            frame_index.extend(list(all_sam))
                            n_frame=K
                    frame_index.sort()
                    #print(frame_index)
                    
                    #  convert the depth sequence to points data
                    #  depth to point
                    #print(n_frame,K)
                    if(1==1):
                        all_frame_3Dpoints_array=np.zeros(shape=[n_frame,fps_sample_num,3])
                        #print(all_frame_3Dpoints_array.shape)    #(20,512,3)
                        i=0
                        for i_frame in frame_index:
                            depthName=imgNames[i_frame]
                            #print(depthName)
                            img_path=os.path.join(pngPath,depthName)
                            #print(img_path)
                            depth_im=load_depth_from_img(img_path)
                            #print(depth_im.shape)   #(424,512)
                            each_frame_points=depth_to_pointcloud(depth_im).T
                            #print(each_frame_points.shape)  #(5130,3)
                            #print(type(each_frame_points))  #<class 'numpy.ndarray'>
                            
                            #Randomly sample 2048 points
                            if (1==1):
                                if len(each_frame_points)<SAMPLE_NUM:   #If the number of points is less than 2048, it will be repeated randomly to 2048
                                    if len(each_frame_points)<SAMPLE_NUM/2:
                                        if len(each_frame_points)<SAMPLE_NUM/4:
                                            #print('4')
                                            rand_points_index=np.random.randint(0,each_frame_points.shape[0],size=SAMPLE_NUM-len(each_frame_points)*4)
                                            #print(len(rand_points_index))
                                            each_frame_points=np.concatenate((each_frame_points,each_frame_points,each_frame_points,each_frame_points,each_frame_points[rand_points_index,:]),axis=0)
                                            #print(each_frame_points.shape)
                                        else:    #4
                                            rand_points_index=np.random.randint(0,each_frame_points.shape[0],size=SAMPLE_NUM-len(each_frame_points)*2)
                                            each_frame_points=np.concatenate((each_frame_points,each_frame_points,each_frame_points[rand_points_index,:]),axis=0)
                                    
                                    else:        #2
                                        #print('2')
                                        rand_points_index=np.random.randint(0,each_frame_points.shape[0],size=SAMPLE_NUM-len(each_frame_points))
                                        each_frame_points=np.concatenate((each_frame_points,each_frame_points[rand_points_index,:]),axis=0)
                                else:    # If it exceeds 2048, randomly sample 2048 points 
                                    rand_points_index=np.random.randint(0,each_frame_points.shape[0],size=SAMPLE_NUM)
                                    each_frame_points=each_frame_points[rand_points_index,:]
                                    #print('3')
                                #print(each_frame_points.shape)
                                #all_frame_4Dpoints_array[i]=each_frame_points
                            if(1==1):     #FPS
                                sampled_idx_l1=farthest_point_sampling_fast(each_frame_points,fps_sample_num)
                                #print(len(sampled_idx_l1))     #512
                                other_idx=np.setdiff1d(np.arange(SAMPLE_NUM),sampled_idx_l1.ravel())
                                #print(len(other_idx))         #512*3=1536
                                new_idx=np.concatenate((sampled_idx_l1.ravel(),other_idx))
                                #print(len(new_idx))           #2048
                                each_frame_points=each_frame_points[new_idx,:]
                            all_frame_3Dpoints_array[i]=each_frame_points[:fps_sample_num,:]
                            #print(all_frame_3Dpoints_array.shape)    #20*512*3
                            i=i+1
                    #Normalized
                    if(1==1):
                        max_x=all_frame_3Dpoints_array[:,:,0].max()
                        max_y=all_frame_3Dpoints_array[:,:,1].max()
                        max_z=all_frame_3Dpoints_array[:,:,2].max()
                        min_x=all_frame_3Dpoints_array[:,:,0].min()
                        min_y=all_frame_3Dpoints_array[:,:,1].min()
                        min_z=all_frame_3Dpoints_array[:,:,2].min()
                        
                        x_len=max_x-min_x
                        y_len=max_y-min_y
                        z_len=max_z-min_z
                        
                        x_center=(max_x+min_x)/2
                        y_center=(max_y+min_y)/2
                        z_center=(max_z+min_z)/2
                        
                        all_frame_3Dpoints_array[:,:,0]=(all_frame_3Dpoints_array[:,:,0]-x_center)/x_len
                        all_frame_3Dpoints_array[:,:,1]=(all_frame_3Dpoints_array[:,:,1]-y_center)/y_len
                        all_frame_3Dpoints_array[:,:,2]=(all_frame_3Dpoints_array[:,:,2]-z_center)/z_len
                        
                        if(video_index==100):
                            print(time.time())
                        print(all_frame_3Dpoints_array.shape)
                        save_npy(all_frame_3Dpoints_array,filename)
                                                                      
                   

 
def save_npy(data, filename):
    file = os.path.join(save_path, filename)
    if not os.path.isfile(file):
        np.save(file, data)

def farthest_point_sampling_fast(pc, sample_num):
    pc_num = pc.shape[0]
    #lx#print(pc_num)
    sample_idx = np.zeros(shape = [sample_num,1], dtype = np.int32)
    sample_idx[0] = np.random.randint(0,pc_num)
    #lx#print(sample_idx.shape)
    #lx#print(sample_idx[0])
    #lx#print(pc[sample_idx[0],:])
    cur_sample = np.tile(pc[sample_idx[0],:], (pc_num,1))
    #lx#print(cur_sample.shape)#lx#(2048, 3)
    diff = pc-cur_sample
    #lx#print(diff.shape)#lx#(2048, 3)
    #lx#print((diff*diff).shape)#lx#(2048, 3)(diff*diff)
    min_dist = (diff*diff).sum(axis = 1)#lx#
    
    #lx#print(min_dist.shape)#lx#(2048,)
    #lx#print(min_dist.reshape(pc_num,1).shape)#lx#(2048,1)
    for cur_sample_idx in range(1,sample_num):#lx#sample_num=512
        ## find the farthest point

        sample_idx[cur_sample_idx] = np.argmax(min_dist)
        # print(sample_idx[cur_sample_idx])
        if cur_sample_idx < sample_num-1:
            diff = pc - np.tile(pc[sample_idx[cur_sample_idx],:], (pc_num,1))
            min_dist = np.concatenate((min_dist.reshape(pc_num,1), (diff*diff).sum(axis = 1).reshape(pc_num,1)), axis = 1).min(axis = 1)  ##?
    #print(min_dist)
    return sample_idx#lx#

def load_depth_from_img(depth_path):
    depth_im = imageio.imread(depth_path) #im is a numpy array
    return depth_im

def depth_to_pointcloud(depth_im):#lx#
    # fx = 2
    # fy = 1
    # cx = 0
    # cy = 0
    #lx#np.array([[0,2,3],[6,5,3]])
    rows,cols = depth_im.shape#lx#(424, 512)
    #lx#print(depth_im.shape)#lx#(2, 3)
    xx,yy = np.meshgrid(range(0,cols), range(0,rows))
    #lx#print(xx)#lx#[[0 1 2],[0 1 2]]
    #lx#print(yy)#lx#[[0 0 0],[1 1 1]]
    valid = depth_im > 0
    #lx#print(valid)#lx#[[False  True  True],[ True  True  True]]
    xx = xx[valid]#lx#
    yy = yy[valid]#lx#
    #lx#print(xx)#lx#[1 2 0 1 2]
    #lx#print(yy)#lx#[0 0 1 1 1]
    depth_im = depth_im[valid]
    #lx#print(depth_im)#lx#[2 3 6 5 3]
    X = (xx - cx) * depth_im / fx#LX#
    Y = (yy - cy) * depth_im / fy#LX#
    #lx#print(X)#lx#[ -6.   -6.  -24.  -15.   -6. ]
    #lx#print(Y)#lx#[ -5.   -7.5 -12.  -10.   -6. ]
    #lx#print((2-4)*8/1)#lx#-6.0
    Z = depth_im
    points3d = np.array([X.flatten(), Y.flatten(), Z.flatten()])
    #lx#print(points3d)#lx#
    #lx#[[ -6.   -6.  -24.  -15.   -6. ]
    #lx# [ -5.   -7.5 -12.  -10.   -6. ]
    #lx# [  2.    3.    6.    5.    3. ]]
    return points3d

In [10]:
if __name__ == '__main__':
    main()


Drink
103


/tmp/ipykernel_320/1761514814.py:202: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  depth_im = imageio.imread(depth_path) #im is a numpy array


(20, 512, 3)
80
(20, 512, 3)
66
(20, 512, 3)
86
(20, 512, 3)
118
(20, 512, 3)
150
(20, 512, 3)
138
(20, 512, 3)
92
(20, 512, 3)
72
(20, 512, 3)
71
(20, 512, 3)
90
(20, 512, 3)
96
(20, 512, 3)
90
(20, 512, 3)
104
(20, 512, 3)
89
(20, 512, 3)
91
(20, 512, 3)
hand_shake
71
(20, 512, 3)
72
(20, 512, 3)
74
(20, 512, 3)
74
(20, 512, 3)
74
(20, 512, 3)
65
(20, 512, 3)
60
(20, 512, 3)
73
(20, 512, 3)
66
(20, 512, 3)
69
(20, 512, 3)
72
(20, 512, 3)
64
(20, 512, 3)
72
(20, 512, 3)
63
(20, 512, 3)
65
(20, 512, 3)
70
(20, 512, 3)
push
74
(20, 512, 3)
70
(20, 512, 3)
61
(20, 512, 3)
64
(20, 512, 3)
79
(20, 512, 3)
58
(20, 512, 3)
64
(20, 512, 3)
45
(20, 512, 3)
64
(20, 512, 3)
52
(20, 512, 3)
70
(20, 512, 3)
54
(20, 512, 3)
68
(20, 512, 3)
67
(20, 512, 3)
38
(20, 512, 3)
58
(20, 512, 3)
sit_down
71
(20, 512, 3)
79
(20, 512, 3)
65
(20, 512, 3)
76
(20, 512, 3)
114
(20, 512, 3)
87
(20, 512, 3)
87
(20, 512, 3)
81
(20, 512, 3)
57
(20, 512, 3)
56
(20, 512, 3)
71
(20, 512, 3)
79
(20, 512, 3)
90
(20, 512, 

In [11]:

fx = 365.481
fy = 365.481
cx = 257.346
cy = 210.347
 

SAMPLE_NUM = 2048
fps_sample_num=512
K = 20  # max frame limit for temporal rank

save_path = '4DV/T'


In [12]:
try:
    os.makedirs(save_path)
except OSError:
    pass

In [13]:
def main():
    data_path = 'NTU_png/test'
    sub_Files = os.listdir(data_path)
    sub_Files.sort()
    #print(sub_Files[1:])
    index17=0

    for s_fileName in sub_Files[1:]:#lx#Traverse 17 settings folders
        index17=index17+1
        #print(s_fileName)
        if(index17>0):
            videoPath = os.path.join(data_path, s_fileName)
            #print(videoPath)
            #videoPath = os.path.join(data_path, s_fileName, 'nturgb+d_depth_masked')
            if os.path.isdir(videoPath):#lx#Determine whether it is a directory
                print(s_fileName)
                video_Files = os.listdir(videoPath)
                video_Files.sort()
                #print(video_Files[1:])
                video_index=0
                video_num=len(video_Files[1:])
                #print(time.time())
                for video_FileName in video_Files[1:]:#lx#Traverse the sample folder
                    video_index=video_index+1
                    #print(video_index)
                    
                    filename=video_FileName+'.npy'
                    file = os.path.join(save_path,filename)  #保存文件名
                    #print(file)
                    pngPath=os.path.join(videoPath,video_FileName)
                    print(pngPath)
                    imgNames=os.listdir(pngPath)
                    imgNames.sort()
                    #print(imgNames)
                    
                    #select a fixed number K of images
                    n_frame=len(imgNames)
                    print(n_frame)
                    all_sam=np.arange(n_frame)
                    #print(all_sam)
                    
                    #Randomly sampled frames   随机采样
                    if(1==0):
                        if n_frame>K:
                            frame_index=random.sample(list(all_sam),K)
                            n_frame=K
                        else:
                            if n_frame<K/2:
                                frame_index=random.sample(list(all_sam),K-n_frame-n_frame)
                                frame_index.extend(list(all_sam))
                                frame_index.extend(list(all_sam))
                                n_frame=K
                            else:
                                frame_index=random.sample(list(all_sam),K-n_frame)
                                frame_index.extend(list(all_sam))
                                n_frame=K
                            frame_index=all_sam.tolist()
                    #Randomly sample frames at equal intervals   stratified sampling分层采样
                    if(1==0):
                        print('now')
                        if n_frame>K:
                            frame_index=[]
                            for jj in range(K):
                                iii=int(np.random.randint(int(n_frame*jj/K),int(n_frame*(jj+1)/K),size=1))
                                frame_index.append(iii)
                            n_frame=K
                        else:
                            frame_index=random.sample(list(all_sam),K-n_frame)
                            frame_index.extend(list(all_sam))
                            n_frame=K
                    #Sampling frames at equal intervals    等间隔采样
                    if(1==1):
                        if n_frame>K:
                            frame_index=[]
                            for jj in range(K):
                                iii= int((int(n_frame*jj/K)+int(n_frame*(jj+1)/K))/2)
                                frame_index.append(iii)
                            n_frame=K
                        else:
                            frame_index=random.sample(list(all_sam),K-n_frame)
                            frame_index.extend(list(all_sam))
                            n_frame=K
                    frame_index.sort()
                    #print(frame_index)
                    
                    #  convert the depth sequence to points data
                    #  depth to point
                    #print(n_frame,K)
                    if(1==1):
                        all_frame_3Dpoints_array=np.zeros(shape=[n_frame,fps_sample_num,3])
                        #print(all_frame_3Dpoints_array.shape)    #(20,512,3)
                        i=0
                        for i_frame in frame_index:
                            depthName=imgNames[i_frame]
                            #print(depthName)
                            img_path=os.path.join(pngPath,depthName)
                            #print(img_path)
                            depth_im=load_depth_from_img(img_path)
                            #print(depth_im.shape)   #(424,512)
                            each_frame_points=depth_to_pointcloud(depth_im).T
                            #print(each_frame_points.shape)  #(5130,3)
                            #print(type(each_frame_points))  #<class 'numpy.ndarray'>
                            
                            #Randomly sample 2048 points
                            if (1==1):
                                if len(each_frame_points)<SAMPLE_NUM:   #If the number of points is less than 2048, it will be repeated randomly to 2048
                                    if len(each_frame_points)<SAMPLE_NUM/2:
                                        if len(each_frame_points)<SAMPLE_NUM/4:
                                            #print('4')
                                            rand_points_index=np.random.randint(0,each_frame_points.shape[0],size=SAMPLE_NUM-len(each_frame_points)*4)
                                            #print(len(rand_points_index))
                                            each_frame_points=np.concatenate((each_frame_points,each_frame_points,each_frame_points,each_frame_points,each_frame_points[rand_points_index,:]),axis=0)
                                            #print(each_frame_points.shape)
                                        else:    #4
                                            rand_points_index=np.random.randint(0,each_frame_points.shape[0],size=SAMPLE_NUM-len(each_frame_points)*2)
                                            each_frame_points=np.concatenate((each_frame_points,each_frame_points,each_frame_points[rand_points_index,:]),axis=0)
                                    
                                    else:        #2
                                        #print('2')
                                        rand_points_index=np.random.randint(0,each_frame_points.shape[0],size=SAMPLE_NUM-len(each_frame_points))
                                        each_frame_points=np.concatenate((each_frame_points,each_frame_points[rand_points_index,:]),axis=0)
                                else:    # If it exceeds 2048, randomly sample 2048 points 
                                    rand_points_index=np.random.randint(0,each_frame_points.shape[0],size=SAMPLE_NUM)
                                    each_frame_points=each_frame_points[rand_points_index,:]
                                    #print('3')
                                #print(each_frame_points.shape)
                                #all_frame_4Dpoints_array[i]=each_frame_points
                            if(1==1):     #FPS
                                sampled_idx_l1=farthest_point_sampling_fast(each_frame_points,fps_sample_num)
                                #print(len(sampled_idx_l1))     #512
                                other_idx=np.setdiff1d(np.arange(SAMPLE_NUM),sampled_idx_l1.ravel())
                                #print(len(other_idx))         #512*3=1536
                                new_idx=np.concatenate((sampled_idx_l1.ravel(),other_idx))
                                #print(len(new_idx))           #2048
                                each_frame_points=each_frame_points[new_idx,:]
                            all_frame_3Dpoints_array[i]=each_frame_points[:fps_sample_num,:]
                            #print(all_frame_3Dpoints_array.shape)    #20*512*3
                            i=i+1
                    #Normalized
                    if(1==1):
                        max_x=all_frame_3Dpoints_array[:,:,0].max()
                        max_y=all_frame_3Dpoints_array[:,:,1].max()
                        max_z=all_frame_3Dpoints_array[:,:,2].max()
                        min_x=all_frame_3Dpoints_array[:,:,0].min()
                        min_y=all_frame_3Dpoints_array[:,:,1].min()
                        min_z=all_frame_3Dpoints_array[:,:,2].min()
                        
                        x_len=max_x-min_x
                        y_len=max_y-min_y
                        z_len=max_z-min_z
                        
                        x_center=(max_x+min_x)/2
                        y_center=(max_y+min_y)/2
                        z_center=(max_z+min_z)/2
                        
                        all_frame_3Dpoints_array[:,:,0]=(all_frame_3Dpoints_array[:,:,0]-x_center)/x_len
                        all_frame_3Dpoints_array[:,:,1]=(all_frame_3Dpoints_array[:,:,1]-y_center)/y_len
                        all_frame_3Dpoints_array[:,:,2]=(all_frame_3Dpoints_array[:,:,2]-z_center)/z_len
                        
                        if(video_index==100):
                            print(time.time())
                        save_npy(all_frame_3Dpoints_array,filename)
                                                                      
                   

 
def save_npy(data, filename):
    file = os.path.join(save_path, filename)
    if not os.path.isfile(file):
        np.save(file, data)

def farthest_point_sampling_fast(pc, sample_num):
    pc_num = pc.shape[0]
    #lx#print(pc_num)
    sample_idx = np.zeros(shape = [sample_num,1], dtype = np.int32)
    sample_idx[0] = np.random.randint(0,pc_num)
    #lx#print(sample_idx.shape)
    #lx#print(sample_idx[0])
    #lx#print(pc[sample_idx[0],:])
    cur_sample = np.tile(pc[sample_idx[0],:], (pc_num,1))
    #lx#print(cur_sample.shape)#lx#(2048, 3)
    diff = pc-cur_sample
    #lx#print(diff.shape)#lx#(2048, 3)
    #lx#print((diff*diff).shape)#lx#(2048, 3)(diff*diff)
    min_dist = (diff*diff).sum(axis = 1)#lx#
    
    #lx#print(min_dist.shape)#lx#(2048,)
    #lx#print(min_dist.reshape(pc_num,1).shape)#lx#(2048,1)
    for cur_sample_idx in range(1,sample_num):#lx#sample_num=512
        ## find the farthest point

        sample_idx[cur_sample_idx] = np.argmax(min_dist)
        # print(sample_idx[cur_sample_idx])
        if cur_sample_idx < sample_num-1:
            diff = pc - np.tile(pc[sample_idx[cur_sample_idx],:], (pc_num,1))
            min_dist = np.concatenate((min_dist.reshape(pc_num,1), (diff*diff).sum(axis = 1).reshape(pc_num,1)), axis = 1).min(axis = 1)  ##?
    #print(min_dist)
    return sample_idx#lx#

def load_depth_from_img(depth_path):
    depth_im = imageio.imread(depth_path) #im is a numpy array
    return depth_im

def depth_to_pointcloud(depth_im):#lx#
    # fx = 2
    # fy = 1
    # cx = 0
    # cy = 0
    #lx#np.array([[0,2,3],[6,5,3]])
    rows,cols = depth_im.shape#lx#(424, 512)
    #lx#print(depth_im.shape)#lx#(2, 3)
    xx,yy = np.meshgrid(range(0,cols), range(0,rows))
    #lx#print(xx)#lx#[[0 1 2],[0 1 2]]
    #lx#print(yy)#lx#[[0 0 0],[1 1 1]]
    valid = depth_im > 0
    #lx#print(valid)#lx#[[False  True  True],[ True  True  True]]
    xx = xx[valid]#lx#
    yy = yy[valid]#lx#
    #lx#print(xx)#lx#[1 2 0 1 2]
    #lx#print(yy)#lx#[0 0 1 1 1]
    depth_im = depth_im[valid]
    #lx#print(depth_im)#lx#[2 3 6 5 3]
    X = (xx - cx) * depth_im / fx#LX#
    Y = (yy - cy) * depth_im / fy#LX#
    #lx#print(X)#lx#[ -6.   -6.  -24.  -15.   -6. ]
    #lx#print(Y)#lx#[ -5.   -7.5 -12.  -10.   -6. ]
    #lx#print((2-4)*8/1)#lx#-6.0
    Z = depth_im
    points3d = np.array([X.flatten(), Y.flatten(), Z.flatten()])
    #lx#print(points3d)#lx#
    #lx#[[ -6.   -6.  -24.  -15.   -6. ]
    #lx# [ -5.   -7.5 -12.  -10.   -6. ]
    #lx# [  2.    3.    6.    5.    3. ]]
    return points3d

In [14]:
if __name__ == '__main__':
    main()


Drink
NTU_png/test/Drink/S001C002P001R001A001
109


/tmp/ipykernel_320/2754938291.py:201: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  depth_im = imageio.imread(depth_path) #im is a numpy array


NTU_png/test/Drink/S001C002P001R002A001
80
NTU_png/test/Drink/S001C002P002R001A001
66
NTU_png/test/Drink/S001C002P002R002A001
85
NTU_png/test/Drink/S001C002P003R001A001
118
hand_shake
NTU_png/test/hand_shake/S001C002P001R001A058
71
NTU_png/test/hand_shake/S001C002P001R002A058
72
NTU_png/test/hand_shake/S001C002P002R001A058
74
NTU_png/test/hand_shake/S001C002P002R002A058
74
push
NTU_png/test/push/S001C002P001R001A052
67
NTU_png/test/push/S001C002P001R002A052
70
NTU_png/test/push/S001C002P002R001A052
61
NTU_png/test/push/S001C002P002R002A052
65
sit_down
NTU_png/test/sit_down/S001C002P001R001A008
71
NTU_png/test/sit_down/S001C002P001R002A008
79
NTU_png/test/sit_down/S001C002P002R001A008
65
NTU_png/test/sit_down/S001C002P002R002A008
76
walk
NTU_png/test/walk/S001C002P001R001A059
99
NTU_png/test/walk/S001C002P001R002A059
132
NTU_png/test/walk/S001C002P002R001A059
94
NTU_png/test/walk/S001C002P002R002A059
98
wear_hat
NTU_png/test/wear_hat/S001C002P001R001A020
111
NTU_png/test/wear_hat/S001C0

In [1]:
import numpy as np
a=np.load('4DV/T/S001C001P001R001A001.npy')
print(a.shape)
print(np.max(a))
print(np.min(a))

(20, 512, 3)
0.5
-0.5000000000000001
